## 2- Data clean and standardize

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import when, regexp_replace, col, trim, to_date

destination_path = '/FileStore/tables/Covid-USA/raw/daily_cases_raw'
spark = SparkSession.builder.appName("COVID Tracking Data - Data cleaning").getOrCreate()
df = spark.read.format("parquet").load(destination_path)
df = df.select("date","state", "positive", "death", "totalTestsViral")

df = df.withColumn("state", trim(df.state))


df = df.withColumn("positive", when(col("positive").isNull(), 0).otherwise(col("positive")))
df = df.withColumn("death", when(col("death").isNull(), 0).otherwise(col("death")))

average_tests = df.selectExpr("avg(totalTestsViral)").collect()[0][0]
df = df.withColumn("totalTestsViral", when(col("totalTestsViral").isNull(), average_tests).otherwise(col("totalTestsViral")))
df = df.withColumn('date', to_date(df['date'], 'yyyyMMdd'))

destination_path = '/FileStore/tables/Covid-USA/bronze/daily_cases_bronze'
df.write.mode('overwrite').format("parquet").save(destination_path)

spark.stop()